In [1]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

Setup Complete


# Context

# Data quality assessment

In [2]:
PATH = 'data/'
sample = pd.read_csv(PATH + 'sample_submission.csv')
test = pd.read_csv(PATH + 'test.csv')
train = pd.read_csv(PATH + 'train.csv')
print(sample)
print(test)
print(train)

       PassengerId  Survived
0           100000         1
1           100001         1
2           100002         1
3           100003         1
4           100004         1
...            ...       ...
99995       199995         1
99996       199996         1
99997       199997         1
99998       199998         1
99999       199999         1

[100000 rows x 2 columns]
       PassengerId  Pclass                Name     Sex   Age  SibSp  Parch  \
0           100000       3    Holliday, Daniel    male  19.0      0      0   
1           100001       3    Nguyen, Lorraine  female  53.0      0      0   
2           100002       1     Harris, Heather  female  19.0      0      0   
3           100003       2        Larsen, Eric    male  25.0      0      0   
4           100004       1       Cleary, Sarah  female  17.0      0      2   
...            ...     ...                 ...     ...   ...    ...    ...   
99995       199995       3       Cash, Cheryle  female  27.0      0      0   
9

# Data exploration

# Summary